In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_wcs(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Existing of a collection
col = client.collections.get("<COLLECTION_NAME>")
if col.exists():
    print("Collection exists")
else:
    print("Collection does not exist")

Collection exists


In [ ]:
# Query an object & its dimensionality
items = client.collections.get("<COLLECTION_NAME>")

data_object = items.query.fetch_object_by_id(
    uuid="UUID", 
    include_vector=True  
)

default_vector = data_object.vector["default"]
dimensionality = len(default_vector)

print(f"The default vector has {dimensionality} dimensions.")

In [ ]:
# Query & Filter by time
from datetime import datetime
from weaviate.classes.query import Filter

collection = client.collections.get("<COLLECTION_NAME>")

filter_time = datetime(2024, 1, 1).isoformat()
print(filter_time)
res = collection.query.fetch_objects(
    filters=Filter.by_creation_time().greater_than(filter_time)
)
print(res)

In [ ]:
# Sorting -> works only with fetch_objects (Similarity searches do not support sorting as they are based sorted by similarity)
from weaviate.classes.query import Sort

movies_col= client.collections.get("<COLLECTION_NAME>")

response = movies_col.query.fetch_objects(
    limit=9,
    sort=Sort.by_property(name="<PROP_NAME>", ascending=False)
)

for o in response.objects:
    print(o.properties)